# Интеллектуальные методы обработки видео

# Ноутбук разделен на два задания:
   * ### [Эвристический SCD](#first)
   * ### [SCD с ML](#second)
   
Сроки выполнения для каждого задания — одна неделя. Оцениваются задания независимо.

## Задание 1. Scene Change Detector
<a id='first'></a>

### Обязательно к прочтению

**Внимание!**

Opencv содержит очень много высокоуровневых функций обработки изображений (например, некоторые алгоритмы компенсации движения, отслеживания объектов, распознавания образов). Использование данной библиотеки в данном задании ограничивается:
* считыванием входного видео
* преобразованием его кадров в другие цветовые пространства
* использованием свёрток Собеля

Использовать библиотеку numpy можно без ограничений.

Если вы хотите использовать функции обработки изображений и видео из другой библиотеки, то оговорите использование этой функции в чате курса.

### Описание входных данных

Выборка для тренировки лежит https://titan.gml-team.ru:5003/sharing/yX8enupJV

Данные о каждом видео лежат в файле *train_dataset\info.json*. Это список из словарей, каждый словарь содержит информацию о расположении видео, о расположении ответов на смены сцен и содержит длину видео

In [ ]:
import numpy as np
import cv2 # Для установки opencv воспользуйтесь командой в терминале conda install -c conda-forge opencv
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

In [ ]:
import json
def load_json_from_file(filename):
    with open(filename, "r") as f:
        return json.load(f, strict=False)


def dump_json_to_file(obj, filename, **kwargs):
    with open(filename, "w") as f:
        json.dump(obj, f, **kwargs)

In [ ]:
video_dataset = load_json_from_file('train_dataset/info.json')
video_dataset

### Загрузка видео ###

Загрузка видео осуществляется при помощи cv2.VideoCapture. Этот код изменять и дописывать не нужно.

In [ ]:
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==False:
            break
        yield frame
    cap.release()

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))

Что такое frames? Это итератор на кадры видео. Чтобы пройтись по всем кадрам последовательности, воспользуйтесь следующей конструкцией:
*Аккуратно, по одной переменной frames можно пройти только один раз!*

In [ ]:
for frame in tqdm(frames):
    pass
for frame in tqdm(frames): # Второй раз уже не будет итерации
    pass

### Пишем свой простой детектор смен сцен

На данном этапе предлагается написать простой Scene Change Detector (SCD) на основе выделения характеристик кадров, подсчёта разницы между кадрами на основе данных характеристик, а также подобрать наиболее оптимальный порог для этих признаков и совместить эти признаки.
Сменой сцен в данной задаче являются только обычные мгновенные смены сцен, без дополнительных эффектов.

В качестве примера приведён простой детектор смен, который считает межкадровую разницу между кадрами.

*Важное замечание. Здесь и далее результатом алгоритма детектора сцен являются **индексы кадров начал сцен**, при этом кадры **нумеруются с 0**. Нулевой кадр в качестве ответа указывать не нужно*

<img src="Hard_cut.jpg">

In [ ]:
def baseline_scene_change_detector(frames, threshold=2000, with_vis=False):
    """
    Baseline SCD

    Arguments:
    frames -- iterator on video frames
    threshold -- parameter of your algorithm (optional)
    with_vis -- saving neighboring frames at a scene change (optional)

    Returns:
    scene_changes -- list of scene changes (idx of frames)
    vis -- list of neighboring frames at a scene change (for visualization)
    metric_values -- list of metric values (for visualization)
    """
    
    def pixel_metric(frame, prev_frame):
        # Базовое расстояние между кадрами - среднеквадратическая ошибка между ними
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)

    scene_changes = []
    vis = []
    metric_values = []
    prev_frame = None
    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        if prev_frame is not None:
            # Находим расстояние между соседними кадрами
            metric_value = pixel_metric(frame, prev_frame)
            if metric_value > threshold:
                scene_changes.append(idx)
                if with_vis:
                    # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                    if len(vis) < 100:
                        vis.append([prev_frame, frame])
            metric_values.append(metric_value)
        else:
            metric_values.append(0)
        prev_frame = frame
    return scene_changes, vis, metric_values

In [ ]:
# frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))
# cuts_base = load_json_from_file(os.path.join('train_dataset', 'gt', '03.json'))['cut']
# scene_changes_base, vis_base, metric_values_base = baseline_scene_change_detector(frames, with_vis=True)

Посмотрим визуально, насколько сильно алгоритм ошибается, а также на значения метрики

In [ ]:
def visualize_metric_error(frame, prev_frame, value):
    fig = plt.figure(figsize=(16,4))
    plt.suptitle('Значение метрики на текущем кадре: {:.4f}'.format(value[0]), fontsize=24)
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(prev_frame[:,:,::-1])
    ax.set_title("Предыдущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(frame[:,:,::-1])
    ax.set_title("Текущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.subplots_adjust(top=0.80)

In [ ]:
idx = 1
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
# смена сцен

In [ ]:
idx = 10
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
# ошибается, это не смена сцен

In [ ]:
def visualize_metric_values(metric_values, threshold, thresh_values, scene_changes, cuts = None):
    sns.set()
    plt.figure(figsize=(16, 8))
    plt.plot(metric_values, label='Значение метрики на кадрах')
    plt.plot(thresh_values, label='Значение порога на кадрах')
    plt.xlabel('Номер кадра')
    plt.ylabel('Значение метрики')
    plt.hlines(y=threshold, xmin=0, xmax=len(metric_values), linewidth=2, color='r', label='Пороговое значение')
    
    if cuts is not None:
        for cut in cuts:
            plt.axvline(x=cut, color='b', linestyle=':', linewidth=0.5, label='Смена сцены')
    if scene_changes is not None:
        for cut in scene_changes:
            plt.axvline(x=cut, color='r', linestyle=':', linewidth=0.5, label='pred')

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

In [ ]:
visualize_metric_values(metric_values_base, 2000, cuts_base)

**Как видим, очень плохо подобран порог, да и сам признак, похоже, сильно зашумлён. Попробуйте что-то своё!**

### Ваше решение

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:
# GRADED CELL: scene_change_detector
# 0.6885444284792273 1000 1500 2500 1000
def scene_change_detector(frames, w1_ = 1000, w2_ = 1500, w3_ = 2500, Threshold=1000, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []
    
    ### START CODE HERE ###
    def get_brightness_hist(frame):
        return cv2.calcHist([cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)], [0], None, [256], [0,256])
    
    def get_color_hist(frame):
        color_hist = []
        for i in range(3):
            color_hist.append(cv2.calcHist([frame], [i], None, [256], [0, 256]))
        return color_hist
    
    def pixel_metric(frame, prev_frame): #mse
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)
    
    def abs_metric(frame, prev_frame):
        return np.sum(np.absolute(frame.astype(np.int32)-prev_frame))
    
    def brightness_metric(frame_hist, prev_frame_hist):
        return 1- (cv2.compareHist(prev_frame_hist, frame_hist, method=cv2.HISTCMP_CORREL)+1) / 2
    
    def color_metric(frame_color_hist, prev_frame_color_hist):      
        color_values = np.zeros(3)
        for i in range(3):
            color_values[i] = (cv2.compareHist(prev_frame_color_hist[i], frame_color_hist[i], method=cv2.HISTCMP_CORREL)+1) / 2.0
        return 1 - np.mean(color_values)
            

    def edge_metric(frame, prev_frame):
        ddepth = cv2.CV_64F
        gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        sobelx1 = cv2.Sobel(gray1, ddepth, 1, 0, ksize=5)
        sobelx2 = cv2.Sobel(gray2, ddepth, 1, 0, ksize=5)
        sobely1 = cv2.Sobel(gray1, ddepth, 0, 1, ksize=5)
        sobely2 = cv2.Sobel(gray2, ddepth, 0, 1, ksize=5)
        
        diff_sobel_x = np.abs(sobelx1 - sobelx2)
        diff_sobel_y = np.abs(sobely1 - sobely2)
        diff_sobel = cv2.addWeighted(diff_sobel_x, 0.5, diff_sobel_y, 0.5, 0)
        
        mse = np.mean(diff_sobel)
        return mse
    def create_edge(frame):
        gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Apply Gaussian blur to reduce noise and smoothen edges 
        blurred1 = cv2.GaussianBlur(src=gray1, ksize=(3, 5), sigmaX=0.5)
        # Perform Canny edge detection 
        edges1 = cv2.Canny(blurred1, 70, 135)    
        return edges1
    def canny_edge(frame_edge, prev_frame_edge):
        return np.mean((frame_edge.astype(np.int32)-prev_frame_edge)**2)
        
    # Ваши внешние переменные
    w1 = w1_
    w2=w2_
    w3=w3_
    # abs
    a_abs = -1
    b_abs = 4
    c_abs = 4
    window_abs = 5
    abs_values = []
    # mse
    a_mse = -1
    b_mse = 4
    c_mse = 4
    window_mse = 5
    mse_values = []
    # bright
    a_bright = -1
    b_bright = 15
    c_bright = 15
    window_bright = 15
    bright_values = []
    # color
    a_color = -1
    b_color = 15
    c_color = 15
    window_color = 15
    color_values = []
    # canny_edge
    a_canny = -1
    b_canny = 2
    c_canny = 8
    window_canny = 10
    canny_values = []
    
    thresh_values = []
    
    OPENCV_METHODS = (
	("Correlation", cv2.HISTCMP_CORREL),
	("Chi-Squared", cv2.HISTCMP_CHISQR),
	("Intersection", cv2.HISTCMP_INTERSECT),
	("Hellinger", cv2.HISTCMP_BHATTACHARYYA))
    prev_frame = None
    prev_frame_hist = None
    prev_frame_color_hist = None
    prev_frame_edge = None
    mse_value = hist_value = color_value = edge_value = 0
    metrics = [
        [mse_value, 0.2],
        [hist_value, 0.35],
        [color_value, 0.375],
        [edge_value, 0.1]
    ]
    ###  END CODE HERE  ###
    
    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        metric_value = 0
        if prev_frame is not None:
            metrics = []
            # MSE
            
            mse_value = pixel_metric(frame, prev_frame)
            m = np.mean(mse_values[max(idx-window_mse, 0):idx])
            std = np.std(mse_values[max(idx-window_mse, 0):idx])
            mse_thresh = a_mse*mse_values[-1]+b_mse*m+std*c_mse
            mse_values.append(mse_value)
            metrics.append((mse_value, mse_thresh))
            
            
            # ABS
            
            abs_value = abs_metric(frame, prev_frame)
            m = np.mean(abs_values[max(idx-window_abs, 0):idx])
            std = np.std(abs_values[max(idx-window_abs, 0):idx])
            abs_thresh = a_abs*abs_values[-1]+b_abs*m+std*c_abs
            abs_values.append(abs_value)
            metrics.append((abs_value, abs_thresh))
            
            # BRIGHTNESSS
            #save histogram of previous frame
            
            frame_hist = get_brightness_hist(frame)
            if prev_frame_hist is None:
                prev_frame_hist = get_brightness_hist(prev_frame)
            bright_value = brightness_metric(frame_hist, prev_frame_hist)
            prev_frame_hist = frame_hist
            m = np.mean(bright_values[max(idx-window_bright, 0):idx])
            std = np.std(bright_values[max(idx-window_bright, 0):idx])
            bright_thresh = a_bright*bright_values[-1]+b_bright*m+std*c_bright
            bright_values.append(bright_value)
            metrics.append((bright_value, bright_thresh))
            
            
            # COLOR
            #save histogram of previous frame
            
            frame_color_hist = get_color_hist(frame)
            if prev_frame_color_hist is None:
                prev_frame_color_hist = get_color_hist(prev_frame)
            color_value = color_metric(frame_color_hist, prev_frame_color_hist)
            prev_frame_color_hist = frame_color_hist
            m = np.mean(color_values[max(idx-window_color, 0):idx])
            std = np.std(color_values[max(idx-window_color, 0):idx])
            color_thresh = a_color*color_values[-1]+b_color*m+std*c_color
            color_values.append(color_value)
            metrics.append((color_value, color_thresh))
            
            
            # CANNY EDGE DETECTION
            
            frame_edge = create_edge(frame)
            if prev_frame_edge is None:
                prev_frame_edge = create_edge(prev_frame)
            canny_edge_value = canny_edge(frame_edge, prev_frame_edge)
            m = np.mean(canny_values[max(idx-window_canny, 0):idx])
            std = np.std(canny_values[max(idx-window_canny, 0):idx])
            canny_thresh = a_canny*canny_values[-1]+b_canny*m+std*c_canny
            canny_values.append(canny_edge_value)
            metrics.append((canny_edge_value, canny_thresh))
            

            # #sliding window with metric values
            # values[:-1] = values[1:]
            # values[-1] = metric_value  
              
            #voting by majority
            votes = 0
            if color_value>color_thresh:
                votes+=2
            if bright_value>bright_thresh:
                votes+=1
            if mse_value>mse_thresh:
                votes+=1
            if abs_value>abs_thresh:
                votes+=1
            if canny_edge_value>canny_thresh:
                votes+=1
            if votes>=4 and idx>2:  
                scene_changes.append(idx)
                if with_vis:
                    # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                    if len(vis) < 100:
                        vis.append([prev_frame, frame])
            metric_values.append([abs_value, mse_value, bright_value, color_value, canny_edge_value])
        else:
            metric_values.append([0,0,0,0,0])
            mse_values.append(0)
            mse_values.append(0)
            abs_values.append(0)
            canny_values.append(0)
            bright_values.append(0)
            color_values.append(0)
        prev_frame = frame

     
         
        ###  END CODE HERE  ###

    return scene_changes, vis, metric_values, thresh_values

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '100.mp4'))
cuts = load_json_from_file(os.path.join('train_dataset', 'gt', '05.json'))['cut']
scene_changes, vis, metric_values, thresh_values = scene_change_detector(frames, with_vis=True)

In [ ]:
metric_values

#### Обратите внимание на скорость работы алгоритма! ####
Если вычислять признаки без циклов по пикселям, а пользоваться методами из numpy, то скорость будет не медленнее 7-8 кадров в секунду.
Например, вы можете использовать функцию `np.histogram` или `cv2.calcHist` для подсчёта гистограмм, а `cv2.Sobel` для применения оператора Собеля к кадру.

In [ ]:
#Посмотрим на найденные смены сцен
idx = -18
visualize_metric_error(vis[idx][0], vis[idx][1], metric_values[scene_changes[idx]])

In [ ]:
print(cuts)
print(scene_changes)

In [ ]:
#Посмотрим на значения метрики
visualize_metric_values(metric_values, 0, thresh_values, scene_changes, cuts)

### Подсчёт метрики F1-Score

Чтобы оценивать алгоритм и научиться сравнивать несколько алгоритмов, нужна метрика качества. В данной задаче для оценки качества алгоритма используется F1-Score. Преимущества использования этой метрики к текущей постановке задачи смены сцен были рассказаны на лекции, напишем только формулы:
$$precision = \frac{tp}{tp+fp}$$
$$recall = \frac{tp}{tp+fn}$$
$$F = 2 * \frac{precision * recall}{precision+recall}$$

На всякий случай опишем как именно происходит подсчёт метрики для видео

1) Сначала из выборки удаляются все кадры, которые по разметке либо являются сложными переходами между сценами, либо помечены как сложные для анализа и разметки (например, титры/обилие компьютерной графики и т.п)


2) Затем для оставшихся кадров уже подсчитывается F1_Score

In [ ]:
#Эти пять клеток кода править не нужно
def calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    predicted_scd = set(predicted_scd)
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    for scd in predicted_scd:
        if scd in true_scd:
            tp += 1
        elif scd not in not_to_use_frames:
            fp += 1
    for scd in true_scd:
        if scd not in predicted_scd:
            fn += 1
    tn = scene_len - len(not_to_use_frames) - tp - fp - fn
    return tp, fp, tn, fn

In [ ]:
def calculate_precision(tp, fp, tn, fn):
    return tp / max(1, (tp + fp))

In [ ]:
def calculate_recall(tp, fp, tn, fn):
    return tp / max(1, (tp + fn))

In [ ]:
def f1_score(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    tp, fp, tn, fn = calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [ ]:
def f1_score_matrix(tp, fp, tn, fn):
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

### Тестируем разработанный метод сразу на нескольких видео 

Проверим, насколько хорошо работает разработанный метод. *Учтите, что итоговое тестирование будет производиться на аналогичном, но недоступном вам наборе видео, но все параметры алгоритмов должны быть указаны вами (иными словами - подобраны на тренировочном наборе).*

In [ ]:
def run_scene_change_detector_all_video(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        # Загружаем видео, его длину и смены сцен
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))
        
        # Составляем список сцен, которые не будут тестироваться
        not_use_frames = set()
        for type_scene_change in ['trash', 'fade', 'dissolve']:
            for bad_scene_range in true_scene_changes.get(type_scene_change, []):
                not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))
        
        predicted_scene_changes, _, _ = scene_change_detector(frames)
        
        param_log['f1_score_{}'.format(video_info['source'])] = f1_score(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
        
        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn 
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log, param_log['_mean_f1_score']

In [ ]:
video_dataset = 'train_dataset'

Данная функция поможет вам посмотреть, на каких видео и на сколько ошибается ваш метод. Прогнать метод на отдельном видео и детально посмотреть кадры вы могли выше.

Кроме того, с помощью этой функции вы можете подобрать оптимальные параметры для метода.

In [ ]:
#Протестируем базовый метод
run_scene_change_detector_all_video(baseline_scene_change_detector, video_dataset)

In [ ]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video(scene_change_detector, video_dataset)

In [ ]:
w1 = [1000, 2000, 3000]
w2 = [500, 1500, 2500]
w3 = [500, 1500, 2500]
threshold = [500, 750, 1000]

for w1_ in w1:
    for w2_ in w2:
        for w3_ in w3:
            for thresh_ in threshold:
                _, score = run_scene_change_detector_all_video(scene_change_detector, video_dataset, w1_,w2_,w3_,thresh_)
                print(score, w1_,w2_, w3_,thresh_)

Когда вы смотрите на результат, обращайте внимание на **_mean_f1_score**  
Именно по этой метрике будет производится финальное оценивание.

## Бонусное задание: распознавание смен сцен типа "наложения"

На практике кроме катов часто встречаются смены сцен, где происходит "наложение" одной сцены на другую:

<img src="Dissolve.jpg">

### Ваше решение 

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector_dissolve(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:
# GRADED CELL: scene_change_detector_dissolve
# 0.6885444284792273 1000 1500 2500 1000
def scene_change_detector_dissolve(frames, w1_=1000, w2_=1500, w3_=2500, Threshold=200, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []

    ### START CODE HERE ###

    def pixel_metric(frame, prev_frame):  # mse
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)

    def pixel_metric_normalized(frame, prev_frame):
        # / pixel_metric(np.full_like(frame, 255),0)
        return pixel_metric(frame, prev_frame) * 0.00004

    def brightness_metric(frame, prev_frame):
        frame_hist = cv2.calcHist(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), [
                                  0], None, [25], [0, 255])
        prev_frame_hist = cv2.calcHist(cv2.cvtColor(
            prev_frame, cv2.COLOR_BGR2GRAY), [0], None, [25], [0, 255])
        hist_value = max(cv2.compareHist(
            prev_frame_hist, frame_hist, method=cv2.HISTCMP_BHATTACHARYYA), 0)
        return hist_value

    def color_metric(frame, prev_frame):
        color_values = np.zeros(3)
        for i in range(3):
            frame_hist = cv2.calcHist(frame, [i], None, [25], [0, 255])
            prev_frame_hist = cv2.calcHist(
                prev_frame, [i], None, [25], [0, 255])
            color_values[i] = max(cv2.compareHist(
                prev_frame_hist, frame_hist, method=cv2.HISTCMP_CORREL), 0)
        return 1 - np.max(color_values)

    def edge_metric(frame, prev_frame):
        ddepth = cv2.CV_64F
        gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        sobelx1 = cv2.Sobel(gray1, ddepth, 1, 0, ksize=5)
        sobelx2 = cv2.Sobel(gray2, ddepth, 1, 0, ksize=5)
        sobely1 = cv2.Sobel(gray1, ddepth, 0, 1, ksize=5)
        sobely2 = cv2.Sobel(gray2, ddepth, 0, 1, ksize=5)

        diff_sobel_x = np.abs(sobelx1 - sobelx2)
        diff_sobel_y = np.abs(sobely1 - sobely2)
        diff_sobel = cv2.addWeighted(diff_sobel_x, 0.5, diff_sobel_y, 0.5, 0)

        mse = np.mean(diff_sobel)
        height, width = frame.shape[:2]
        # print(height, width)
        return mse / (height*width)

    # Ваши внешние переменные
    w1 = w1_
    w2 = w2_
    w3 = w3_
    OPENCV_METHODS = (
	("Correlation", cv2.HISTCMP_CORREL),
	("Chi-Squared", cv2.HISTCMP_CHISQR),
	("Intersection", cv2.HISTCMP_INTERSECT),
	("Hellinger", cv2.HISTCMP_BHATTACHARYYA))
    prev_frame = None
    prev_prev_frame = None
    mse_thresh = 0.03
    ###  END CODE HERE  ###

    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        if prev_frame is not None:
            if prev_prev_frame is not None:
                # mse
                # mse_value = pixel_metric_normalized(frame, prev_frame)
                # # histograms
                # hist_value = brightness_metric(frame, prev_frame)
                # color_value = color_metric(frame, prev_frame)
                # sobel
                # edge_value = edge_metric(frame, prev_frame)
                # weights
                # w = 1000
                w1 = 2000
                # w2 = 1500
                # w3 = 1000
                # w4 = 10000 #lol
                # if (mse_value <= mse_thresh):
                #     hist_value = 0
                #     color_value = 0
                sum = cv2.addWeighted(frame, 0.5, prev_prev_frame, 0.5, 0)
                
                metric_value = pixel_metric_normalized(sum, prev_frame)*w1
                if metric_value > Threshold:
                    scene_changes.append(idx)
                    if with_vis:
                        # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                        if len(vis) < 100:
                            vis.append([prev_frame, frame])
                metric_values.append(metric_value)
            else:
                metric_values.append(0)
            prev_prev_frame = prev_frame
        prev_frame = frame

        ###  END CODE HERE  ###

    return scene_changes, vis, metric_values

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))
cuts = load_json_from_file(os.path.join('train_dataset', 'gt', '03.json'))['dissolve']
scene_changes, vis, metric_values = scene_change_detector_dissolve(frames, with_vis=True)

In [ ]:
#Посмотрим на найденные смены сцен
idx = 3
visualize_metric_error(vis[idx][0], vis[idx][1], metric_values[scene_changes[idx]])

In [ ]:
#Посмотрим на значения метрики
visualize_metric_values(metric_values, 750, cuts)

В качестве метрики качества используется видоизменённый f1-score:

Так как смена сцен не происходит за один кадр, попаданием считается попадание ответа смены сцен в отрезок, где происходит наложение.  
**Обратите внимание**, что несколько раз указывать одну смену сцен не нужно.

Попадание вне отрезков смен сцен путём наложения считается как false positive, не попадание в указанный отрезок - как false negative

In [ ]:
#Эти три клетки кода править не нужно
def calculate_matrix_dissolve(true_scd, predicted_scd, scene_len):
    predicted_scd = set(predicted_scd)
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    checked_dissolve_segments = set()
    total_scene_dissolve_len = np.sum([dissolve_segment[1] - dissolve_segment[0] + 1 for dissolve_segment in true_scd])
    for scd in predicted_scd:
        for dissolve_segment in true_scd:
            if scd in range(dissolve_segment[0], dissolve_segment[1] + 1):
                if tuple(dissolve_segment) not in checked_dissolve_segments:
                    tp += 1
                    checked_dissolve_segments.add(tuple(dissolve_segment))
                break
        else:
            fp += 1
    fn = len(true_scd) - len(checked_dissolve_segments)
    tn = scene_len - total_scene_dissolve_len + len(true_scd) - tp - fp - fn
    return tp, fp, tn, fn

In [ ]:
def f1_score_dissolve(true_scd, predicted_scd, scene_len):
    tp, fp, tn, fn = calculate_matrix_dissolve(true_scd, predicted_scd, scene_len)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [ ]:
def run_scene_change_detector_all_video_dissolve(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))
        
        predicted_scene_changes, _, _ = scene_change_detector(frames)
        param_log['f1_score_{}'.format(video_info['source'])] = f1_score_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log

In [ ]:
video_dataset_path = 'train_dataset'

In [ ]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video_dissolve(scene_change_detector_dissolve, video_dataset_path)

### Немного об оценивании задания

Оценивание задания будет производиться по следующей схеме:  

Пусть на скрытой выборке по F-метрике вы получили X, лучшее решение получило Y.

1. Базовая часть оценивется как $$20 * \left(\frac{\max(0, X_{base} - 0.5)}{Y_{base} - 0.5}\right)^2 + Bonus_{base}$$ Бонусные баллы $Bonus$ можно получить за оригинальные идеи в задаче или в её реализации
2. Дополнительное задание оценивается как $$5 * \frac{\max(0, X_{add} - 0.1)}{Y_{add} - 0.1} + Bonus_{add}$$Процесс получения бонусных баллов аналогичен получению бонусных баллов в базовой части

### Ваши ощущения ##

*До дедлайна пару часов и вы никак не можете улучшить текущее решение? Или наоборот, вы всё сделали очень быстро? Опишите кратко ваши ощущения от задания - сколько времени вы потратили на задание, сколько вы потратили на изучение питона и установку необходимых библиотек, как быстро вы придумывали новые идеи и как они давали прирост по метрике и в целом насколько это задание вам понравилось и что хотели бы изменить/добавить.*

<a id='second'></a>

## Задание 2. Scene change detector. Машинное обучение

**Внимание!**

В этом задании можно использовать все, что разрешалось в Задании №1, а также библиотеки:
* pandas
* sklearn

Большинство функций, использующихся в этом задании, реализованы выше.

### Бейзлайн

In [ ]:
from sklearn.svm import SVC
import pandas as pd
import pickle

Обучим простой SVM классификатор над метрикой попиксельной разницы кадров на нескольких видео. Воспользуемся функцией из первого задания

In [ ]:
def get_train_data(train_videos):
    X_train, y_train = None, np.array([])
    for video in train_videos:
        frames = read_video(os.path.join('train_dataset', 'video', f'{video}.mp4'))
        # baseline функция попиксельного сравнения кадров из прошлого задания
        # нам нужны не сами смены сцен, а только значения метрик
        cuts, _, metric_values = scene_change_detector(frames) 
        
        video_scenes = np.array([0 for i in range(len(metric_values))])
        video_scenes[cuts] += 1
        
        # добавляем в разметку текущее видео
        if X_train is None:
            X_train = metric_values
        else:
            X_train = np.concatenate((X_train, metric_values),0)
        y_train = np.hstack((y_train, video_scenes))
        
    return X_train, y_train 

In [ ]:
A = [[0,0,0,0,0],[0,0,0,0,0]]
B = [[0,0,0,0,0],[0,0,0,0,0]]
np.concatenate((A,B),0)

In [ ]:
train_videos = ['04', '05']

X_train, y_train = get_train_data(train_videos)
X_train, y_train

In [ ]:
# создание модели
# подберите лучшие параметры для данной задачи
params = {"kernel": "rbf", "C": 1}
model = SVC(**params)
model.fit(X_train.reshape(-1, 1), y_train)

 Сохраним модель в файле *model.pkl*

In [ ]:
pickle.dump(model, open("model.pkl", "wb"))

Посмотрим как модель работает на тестовых видео

Обратите внимание на то, что внутри функции модель загружается из памяти из файла *model.pkl*

In [ ]:
def baseline_scene_change_detection_ml(frames):
    # подготавливаем данные для видео
    _, _, metric_values = baseline_scene_change_detector(frames) 
    X_test = np.array(metric_values).reshape(-1, 1)
    # загружаем модель и делаем предсказания
    model = pickle.load(open("model.pkl", 'rb')) 
    predict_cuts = model.predict(X_test)
    
    return np.where(predict_cuts > 0)[0], [], metric_values

In [ ]:
def run_scene_change_detector_ml_one_video(scene_change_detector, dataset_path, video_num):
    video_info = load_json_from_file(os.path.join(dataset_path, 'info.json'))[video_num]
    
    # Загружаем видео, его длину и смены сцен
    frames = read_video(os.path.join(dataset_path, video_info['source']))
    video_len = video_info['len']
    true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))

    # Составляем список сцен, которые не будут тестироваться
    not_use_frames = set()
    for type_scene_change in ['trash', 'fade', 'dissolve']:
        for bad_scene_range in true_scene_changes.get(type_scene_change, []):
            not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))

    predicted_scene_changes, _, _ = scene_change_detector(frames)

    return f1_score(
        true_scene_changes['cut'],
        predicted_scene_changes,
        video_len,
        not_use_frames
    )

Посчитаем F1 score для одного видео:

In [ ]:
video_num = 1
run_scene_change_detector_ml_one_video(baseline_scene_change_detection_ml, 'train_dataset', video_num)

In [ ]:
run_scene_change_detector_all_video(baseline_scene_change_detection_ml, video_dataset_path)

### Ваше решение

Чтобы использовать свою обученную модель при отправке решения, необходимо сохранить ее через пакет pickle в файл model.pkl и отправить его вместе с jupyter ноутбуком.
Этот файл вы можете открывать и использовать прямо в функции вашего решения

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector_dissolve(frames)`  
* Строку (# GRADED CELL: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:

# GRADED CELL: scene_change_detector_ml

def scene_change_detector_ml(frames, with_vis = False):
    scene_changes = []
    vis = []
    metric_values = []
    
    ###
    def get_brightness_hist(frame):
        return cv2.calcHist([cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)], [0], None, [256], [0,256])
    
    def get_color_hist(frame):
        color_hist = []
        for i in range(3):
            color_hist.append(cv2.calcHist([frame], [i], None, [256], [0, 256]))
        return color_hist
    
    def pixel_metric(frame, prev_frame): #mse
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)
    
    def abs_metric(frame, prev_frame):
        return np.sum(np.absolute(frame.astype(np.int32)-prev_frame))
    
    def brightness_metric(frame_hist, prev_frame_hist):
        return 1- (cv2.compareHist(prev_frame_hist, frame_hist, method=cv2.HISTCMP_CORREL)+1) / 2
    
    def color_metric(frame_color_hist, prev_frame_color_hist):      
        color_values = np.zeros(3)
        for i in range(3):
            color_values[i] = (cv2.compareHist(prev_frame_color_hist[i], frame_color_hist[i], method=cv2.HISTCMP_CORREL)+1) / 2.0
        return 1 - np.mean(color_values)
    def create_edge(frame):
        gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Apply Gaussian blur to reduce noise and smoothen edges 
        blurred1 = cv2.GaussianBlur(src=gray1, ksize=(3, 5), sigmaX=0.5)
        # Perform Canny edge detection 
        edges1 = cv2.Canny(blurred1, 70, 135)    
        return edges1
    def canny_edge(frame_edge, prev_frame_edge):
        return np.mean((frame_edge.astype(np.int32)-prev_frame_edge)**2)      
    ###
    
    
    ### START CODE HERE ###
    # Ваши внешние переменные
    # abs
    a_abs = -1
    b_abs = 4
    c_abs = 4
    window_abs = 5
    abs_values = []
    # mse
    a_mse = -1
    b_mse = 4
    c_mse = 4
    window_mse = 5
    mse_values = []
    # bright
    a_bright = -1
    b_bright = 15
    c_bright = 15
    window_bright = 15
    bright_values = []
    # color
    a_color = -1
    b_color = 15
    c_color = 15
    window_color = 15
    color_values = []
    # canny_edge
    a_canny = -1
    b_canny = 2
    c_canny = 8
    window_canny = 10
    canny_values = []
    metrics=[]
    prev_frame = None
    prev_frame_hist = None
    prev_frame_color_hist = None
    prev_frame_edge = None
    mse_value = hist_value = color_value = edge_value = 0
    ###  END CODE HERE  ###
    
    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        
        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        # COLOR
        #save histogram of previous frame
        if prev_frame is not None:
            metrics = []
            # MSE
            
            mse_value = pixel_metric(frame, prev_frame)
            m = np.mean(mse_values[max(idx-window_mse, 0):idx])
            std = np.std(mse_values[max(idx-window_mse, 0):idx])
            mse_thresh = a_mse*mse_values[-1]+b_mse*m+std*c_mse
            mse_values.append(mse_value)
            metrics.append((mse_value, mse_thresh))
            
            
            # ABS
            
            abs_value = abs_metric(frame, prev_frame)
            m = np.mean(abs_values[max(idx-window_abs, 0):idx])
            std = np.std(abs_values[max(idx-window_abs, 0):idx])
            abs_thresh = a_abs*abs_values[-1]+b_abs*m+std*c_abs
            abs_values.append(abs_value)
            metrics.append((abs_value, abs_thresh))
            
            # BRIGHTNESSS
            #save histogram of previous frame
            
            frame_hist = get_brightness_hist(frame)
            if prev_frame_hist is None:
                prev_frame_hist = get_brightness_hist(prev_frame)
            bright_value = brightness_metric(frame_hist, prev_frame_hist)
            prev_frame_hist = frame_hist
            m = np.mean(bright_values[max(idx-window_bright, 0):idx])
            std = np.std(bright_values[max(idx-window_bright, 0):idx])
            bright_thresh = a_bright*bright_values[-1]+b_bright*m+std*c_bright
            bright_values.append(bright_value)
            metrics.append((bright_value, bright_thresh))
            
            
            # COLOR
            #save histogram of previous frame
            
            frame_color_hist = get_color_hist(frame)
            if prev_frame_color_hist is None:
                prev_frame_color_hist = get_color_hist(prev_frame)
            color_value = color_metric(frame_color_hist, prev_frame_color_hist)
            prev_frame_color_hist = frame_color_hist
            m = np.mean(color_values[max(idx-window_color, 0):idx])
            std = np.std(color_values[max(idx-window_color, 0):idx])
            color_thresh = a_color*color_values[-1]+b_color*m+std*c_color
            color_values.append(color_value)
            metrics.append((color_value, color_thresh))
            
            
            # CANNY EDGE DETECTION
            
            frame_edge = create_edge(frame)
            if prev_frame_edge is None:
                prev_frame_edge = create_edge(prev_frame)
            canny_edge_value = canny_edge(frame_edge, prev_frame_edge)
            m = np.mean(canny_values[max(idx-window_canny, 0):idx])
            std = np.std(canny_values[max(idx-window_canny, 0):idx])
            canny_thresh = a_canny*canny_values[-1]+b_canny*m+std*c_canny
            canny_values.append(canny_edge_value)
            metrics.append((canny_edge_value, canny_thresh))
            

            # #sliding window with metric values
            # values[:-1] = values[1:]
            # values[-1] = metric_value  
              
            #voting by majority
            votes = 0
            if color_value>color_thresh:
                votes+=2
            if bright_value>bright_thresh:
                votes+=1
            if mse_value>mse_thresh:
                votes+=1
            if abs_value>abs_thresh:
                votes+=1
            if canny_edge_value>canny_thresh:
                votes+=1
            if votes>=4 and idx>2:  
                metric_values.append(1)   
                scene_changes.append(idx)
            else:
                metric_values.append(0)
        else:
            metric_values.append(0)
            mse_values.append(0)
            mse_values.append(0)
            abs_values.append(0)
            canny_values.append(0)
            bright_values.append(0)
            color_values.append(0)
        prev_frame = frame


        ###  END CODE HERE  ###
    
    model = pickle.load(open("model.pkl", 'rb')) 
    X_test = np.array(metric_values).reshape(-1, 1)  
    predict_cuts = model.predict(X_test)
    
    return np.where(predict_cuts == 1)[0], vis, metric_values

In [ ]:
from sklearn.model_selection import cross_val_score 
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys

In [ ]:
train_videos = ['04','05']

X_train, y_train = get_train_data(train_videos)
# X,y = balanced_subsample(x,y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 45)
print(X_train.shape, y_train.shape)

In [ ]:
X_train

In [ ]:
model = SVC()
model.fit(X_train, y_train)
prediction = model.predict(X_train)
print(sklearn.metrics.classification_report(y_train,prediction))

In [ ]:

param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2, scoring='f1')
grid.fit(X_train,y_train)


In [ ]:
grid.best_params_

In [ ]:
predic = grid.predict(X_train.reshape(-1,1))
print(sklearn.metrics.classification_report(y_train,predic))

In [ ]:
p = {'C': 1, 'gamma': 1, 'kernel': 'linear'}



model = SVC(**p)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
Y_pred = model.predict(X_train.reshape(-1,1))
print(sklearn.metrics.classification_report(y_train, Y_pred))

In [ ]:
pickle.dump(model, open("model.pkl", "wb"))

Проверим ваше решение на всех видео.

Не забывайте о том, что при итоговой оценке решений будет использоваться другой набор видео. Не переобучите модель!

In [ ]:
video_dataset_path = 'train_dataset'

In [ ]:
video_num = 1
run_scene_change_detector_ml_one_video(scene_change_detector_ml, 'train_dataset', video_num)

In [ ]:
run_scene_change_detector_all_video(scene_change_detector_ml, video_dataset_path)

**Советы**

* Используйте кросс-валидацию
* Подумайте как лучше разделять видео на тренировочную и тестовые выборки
* Подбирайте параметры модели (в библиотеке sklearn есть метод GridSearchCV для автоматического подбора параметров)
* Пробуйте разные методы машинного обучения (из sklearn)

## Бонусное задание: детектор смен сцен типа наложение

Аналогично детектору из задания №1 за исключением того, что можно (и нужно) использовать машинное обучение:)

In [ ]:
# GRADED CELL: scene_change_detector_dissolve_ml

def scene_change_detector_dissolve_ml(frames, threshold=None, with_vis=False):
    scene_changes = []
    vis = []
    metric_values = []
    
    ### START CODE HERE ###
    # Ваши внешние переменные
    ###  END CODE HERE  ###
    
    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        
        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        ###  END CODE HERE  ###

    return scene_changes, vis, metric_values

In [ ]:
video_dataset_path = 'train_dataset'
#Протестируем разработанный вами метод
run_scene_change_detector_all_video_dissolve(scene_change_detector_dissolve, video_dataset_path)

### Ваши ощущения ##

*Как и в первой части интересно узнать какие моменты показались простыми, а какие сложными. Много ли времени ушло на изучение sklearn и методов машинного обучения. Дало ли машинное обучение сразу прирост по сравнению с эврестической частью? Как вы контролировали переобучение?*